# Calculate important values from NP trajectories

Cal Craven
08-06-20
Goal- Use MDanalysis to look at important characteristics of single np's
1. Read in DCD data
2. Calculate Radius of Gyration and Asphericity
3. Correlation Matrix

## Need to read in files from entire single NP list and then make plots for them

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import os.path as path
import scipy
from readtraj import *
from collections import Counter
from plotting import *
import MDAnalysis as md
import random

In [3]:
#Read in Data using xyz readers
index=0
scale=0.395
data=XYZ_To_NParray("../1bb21ced4538fec0a1744ab973600ca0/COG.xyz")*scale 

In [4]:
#Calculate Radius of Gyration

In [5]:
folder = '../copy_over_figures/044769366b1fef48f7789764c3740fd6'
dcdpath = folder + '/tnp.dcd'
toppath = folder + '/tnp.hoomdxml'
universe = md.Universe(toppath,dcdpath,topology_format='XML')

In [ ]:
#df=pandas.read_csv('Statepoints.csv')

In [2]:
with open('file_list') as file:
    filelist = file.readlines()
    file.close()
print(filelist)

['1499538af491ce89bbfed0de287077ee/\n', '14a2b8b463e21ae1954f5a1c617847e2/\n', '1bb21ced4538fec0a1744ab973600ca0/\n', '1bd06d14f87674a7d263b6ea42872fb9/\n', '270c0867ad60b13942fa6ae00f91bf96/\n', '2786e7ea28a7e59e2e50ff5e580c7d3e/\n', '2c8bf79a2c578f2bec76c078865b4132/\n', '33c0d152f8ff745cc703b13fe4967842/\n', '3688f57faae97574bce28b1c7ff935bd/\n', '3719d43c55829a655347311220b2d273/\n', '46d076c0ba37bdefdfa286d61e4aaa20/\n', '4e58830be1dccdf9c2b856c589273904/\n', '50576410dd0dfb272b45f35be32e0118/\n', '5a3d11f1fd9ab9924c570969af138c29/\n', '5e5469e3ad539a2a44caffb514c3a1b0/\n', '662abffa43a82e398b60d79bb4ddaa22/\n', '763756f52444966ccdc515902ae14a08/\n', '78cfaea4bd511bd0d0f48dbdb39b0b3a/\n', '792c384b931ef2daa526ebda17b63e60/\n', '7b1a85b820a3f516ffaa29d15fa65bf5/\n', '7ca78ebf9f2907ea04f44bb7b19bc127/\n', '8276a6fd578140ddaa6b6edc6c1d674d/\n', '83adca02aebb6f5634d5a92548ab70be/\n', '88654cf966042dfcd059a45527c3fe8b/\n', '889309954aa3a0fc4802c5ca3dd108ed/\n', '8f983bb02736efd50ba4e21

In [45]:
for folder in filelist:
    toppath =  '../' + folder[:-2] + '/tnp-box.hoomdxml'
    newtoppath=  '../' + folder[:-2] + '/update_tnp.hoomdxml'
    with open(toppath,'r') as myrfile:
        lines = myrfile.readlines()
        open(newtoppath, 'w').close()
        lines[2]=lines[2][:28]+lines[4][23:39]+'\n'
        print(folder,lines[2])
        with open(newtoppath,'w') as mywfile:
            for line in lines:
                mywfile.write(line)
            mywfile.close()

1499538af491ce89bbfed0de287077ee/
 <configuration time_step="0" natoms="20325">

14a2b8b463e21ae1954f5a1c617847e2/
 <configuration time_step="0" natoms="30075">

1bb21ced4538fec0a1744ab973600ca0/
 <configuration time_step="0" natoms="17025">

1bd06d14f87674a7d263b6ea42872fb9/
 <configuration time_step="0" natoms="29625">

270c0867ad60b13942fa6ae00f91bf96/
 <configuration time_step="0" natoms="33825">

2786e7ea28a7e59e2e50ff5e580c7d3e/
 <configuration time_step="0" natoms="31650">

2c8bf79a2c578f2bec76c078865b4132/
 <configuration time_step="0" natoms="34725">

33c0d152f8ff745cc703b13fe4967842/
 <configuration time_step="0" natoms="22425">

3688f57faae97574bce28b1c7ff935bd/
 <configuration time_step="0" natoms="26775">

3719d43c55829a655347311220b2d273/
 <configuration time_step="0" natoms="30225">

46d076c0ba37bdefdfa286d61e4aaa20/
 <configuration time_step="0" natoms="30525">

4e58830be1dccdf9c2b856c589273904/
 <configuration time_step="0" natoms="29475">

50576410dd0dfb272b45f35be32e

In [152]:
full_r_g=[]
chain_r_g=[]
asphere=[]
for folder in filelist:
    dcdpath = '../' + folder[:-2] + '/assemble.dcd'
    toppath =  '../' + folder[:-2] + '/update_tnp.hoomdxml'
    statepath = '../' + folder[:-2] +'/signac_statepoint.json'
    print(folder)
    if path.isfile(dcdpath) and path.isfile(toppath):
        universe = md.Universe(toppath,dcdpath,topology_format='XML')
        rand_np=random.randrange(24)
        atom_length = int(len(universe.atoms)/25)
        chains_ids=[str(atom_length*rand_np+154),str(atom_length*(1+rand_np))]
        full_ids=[str(atom_length*rand_np+1),str(atom_length*(1+rand_np))]
        with open(statepath,'r') as myrfile:
            lines = myrfile.readlines()
            chainlength = int(lines[1][17:19])
        atoms = universe.select_atoms("bynum "+chains_ids[0]+':'+chains_ids[1])
        total=0
        for chain_id in np.arange(0,len(atoms)/chainlength*3,1):
            chain=atoms[int(chain_id*(chainlength/3)):int((chain_id+1)*chainlength/3)]
            total += chain.radius_of_gyration()
        average = total/(len(atoms)/chainlength*3)
        chain_r_g.append(average)
        full_r_g.append(universe.atoms.select_atoms('bynum ' +full_ids[0]+':'+full_ids[1]).radius_of_gyration())
        asphere.append(universe.atoms.asphericity())
#df = pandas.DataFrame({'Radius of Gyration (nm)': r_g,
#                       'Asphericity': asphere})


1499538af491ce89bbfed0de287077ee/

14a2b8b463e21ae1954f5a1c617847e2/

1bb21ced4538fec0a1744ab973600ca0/

1bd06d14f87674a7d263b6ea42872fb9/

270c0867ad60b13942fa6ae00f91bf96/

2786e7ea28a7e59e2e50ff5e580c7d3e/

2c8bf79a2c578f2bec76c078865b4132/

33c0d152f8ff745cc703b13fe4967842/

3688f57faae97574bce28b1c7ff935bd/

3719d43c55829a655347311220b2d273/

46d076c0ba37bdefdfa286d61e4aaa20/

4e58830be1dccdf9c2b856c589273904/

50576410dd0dfb272b45f35be32e0118/

5a3d11f1fd9ab9924c570969af138c29/

5e5469e3ad539a2a44caffb514c3a1b0/

662abffa43a82e398b60d79bb4ddaa22/

763756f52444966ccdc515902ae14a08/

78cfaea4bd511bd0d0f48dbdb39b0b3a/

792c384b931ef2daa526ebda17b63e60/

7b1a85b820a3f516ffaa29d15fa65bf5/

7ca78ebf9f2907ea04f44bb7b19bc127/

8276a6fd578140ddaa6b6edc6c1d674d/

83adca02aebb6f5634d5a92548ab70be/

88654cf966042dfcd059a45527c3fe8b/

889309954aa3a0fc4802c5ca3dd108ed/

8f983bb02736efd50ba4e2189f81013d/

912249a3539a9cfda711eccf7a00a8c6/

91a98aae5ae1683ed1c68e98549f7120/

957f3500daae01480638

In [153]:
df = pandas.DataFrame({'Graft_rg': 2.5/np.array(chain_r_g),
                       'NP_rg': 2.5/np.array(full_r_g),
                       'Asphericity': asphere})

In [155]:
df.to_csv('Bulk_sim_rg_asphere.csv')